In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
train_dir = "./dataset/asl_alphabet_train/asl_alphabet_train"  
img_size = (128, 128) 
batch_size = 32 
total_epochs = 40 
chunk_size = 5
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2 
)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation"
)

class_labels = list(train_data.class_indices.keys())
print("Class Labels:", class_labels)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Class Labels: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [3]:
checkpoint_path = "asl_model.h5"

if os.path.exists(checkpoint_path):
    print("\n🔄 Loading previous checkpoint...\n")
    model = load_model(checkpoint_path)
else:
    print("\n🚀 Creating a new model...\n")
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(128, 128, 3),  
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = False 

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(len(train_data.class_indices), activation="softmax")
    ])

    model.compile(optimizer="adam",
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])


🚀 Creating a new model...

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
checkpoint = ModelCheckpoint(
    checkpoint_path,  
    monitor="val_accuracy", 
    save_best_only=True,
    verbose=1
)

In [ ]:
for i in range(0, total_epochs, chunk_size):
    print(f"\n🚀 Training epochs {i+1} to {i+chunk_size}...\n")

    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=chunk_size,  # train for only a few epochs
        steps_per_epoch=len(train_data) // 2,
        callbacks=[checkpoint]
    )


🚀 Training epochs 1 to 5...

Epoch 1/5
  81/1087 ━━━━━━━━━━━━━━━━━━━━ 40:03 2s/step - accuracy: 0.9292 - loss: 0.2386